In [3]:
import os
import pyspark
import findspark
findspark.init() 
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [4]:
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))
print("PATH:", os.environ.get("PATH"))

from pyspark import SparkContext
try:
    sc = SparkContext.getOrCreate()
    print("SparkContext initialized successfully!")
except Exception as e:
    print(f"Error initializing SparkContext: {e}")

JAVA_HOME: /usr/lib/jvm/java-11-openjdk-amd64
PATH: /home/joaquin_l_calvo/.virtualenvs/data_mining/bin:/home/joaquin_l_calvo/.vscode-server/bin/f1a4fb101478ce6ec82fe9627c43efbf9e98c813/bin/remote-cli:/usr/lib/jvm/java-11-openjdk-amd64/bin:/usr/local/hadoop/bin:/usr/local/hadoop/sbin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Program Files/Common Files/Oracle/Java/javapath:/mnt/c/Program Files (x86)/Common Files/Oracle/Java/java8path:/mnt/c/Program Files (x86)/Common Files/Oracle/Java/javapath:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0/:/mnt/c/Windows/System32/OpenSSH/:/mnt/c/Program Files/dotnet/:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/WINDOWS/system32:/mnt/c/WINDOWS:/mnt/c/WINDOWS/System32/Wbem:/mnt/c/WINDOWS/System32/WindowsPowerShell/v1.0/:/mnt/c/WINDOWS/System32/OpenSSH/:/mnt/c/Program Files/Microsoft/Azure Functions

your 131072x1 screen size is bogus. expect trouble
24/12/01 17:25:58 WARN Utils: Your hostname, DESKTOP-LQJ6T08 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/01 17:25:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/01 17:25:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkContext initialized successfully!


In [5]:
from pyspark.sql.dataframe import DataFrame
from typing import Dict, List

def read_from_db(spark, config: Dict[str, str], query: str) -> DataFrame:
    """
    Reads data from an Oracle database into a Spark DataFrame.

    Args:
        spark (SparkSession): Pre-created SparkSession object.
        config (Dict[str, str]): Database configuration dictionary with keys `db_user`, `db_password`, and `db_dsn`.
        query (str): SQL query to execute.

    Returns:
        DataFrame: Spark DataFrame containing the query result.
    """
    url = f"jdbc:oracle:thin:@{config['db_dsn']}?ssl=true"

    
    return spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", f"({query})") \
        .option("user", config["db_user"]) \
        .option("password", config["db_password"]) \
        .option("driver", "oracle.jdbc.OracleDriver") \
        .load()

In [6]:
import yaml

def load_config(file_path):
    """
    Loads a YAML configuration file and returns the contents as a dictionary.
    
    :param file_path: Path to the YAML file.
    :return: Dictionary with configuration values.
    """
    try:
        with open(file_path, 'r') as file:
            config = yaml.safe_load(file)
            return config
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except yaml.YAMLError as e:
        print(f"Error parsing YAML file: {e}")
        return None


In [12]:
from pyspark.sql import SparkSession
from pathlib import Path

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read Utility Matrix") \
    .master("local[*]") \
    .config("spark.driver.memory", "2G") \
    .config("spark.jars", "/usr/local/spark/jars/ojdbc17.jar") \
    .getOrCreate()
    

# Dynamically construct the path to 'config.yaml'
config_file_path = Path.home() / "Trento" / "Data_Mining" / "Data-Mining-Project" / "src" / "config.yaml"

# Load database configuration
config = load_config(config_file_path)

# Read utility matrix from the database
print("Verifying utility matrix in the database...")
try:
    result_df = read_from_db(spark, config, "SELECT * FROM interactions")
    result_df.show(10, truncate=False)
except Exception as e:
    print(f"Error reading from the database: {e}")

# Optional: Convert to Pandas for further analysis
pandas_df = result_df.toPandas()
print(pandas_df.head())

Verifying utility matrix in the database...
Database URL: jdbc:oracle:thin:@(description=  (retry_count=20)  (retry_delay=3)  (address=    (protocol=tcps)    (port=1522)    (host=adb.eu-milan-1.oraclecloud.com)  )  (connect_data=    (service_name=g126d38945ee7b1_w1d1rk5b8lafcoir_medium.adb.oraclecloud.com)  )  (security=    (ssl_server_dn_match=yes)  ))?TNS_ADMIN=/path/to/wallet
Error reading from the database: An error occurred while calling o64.load.
: java.lang.ClassNotFoundException: oracle.jdbc.OracleDriver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptio

NameError: name 'result_df' is not defined

24/12/01 18:46:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 354127 ms exceeds timeout 120000 ms
24/12/01 18:46:54 WARN SparkContext: Killing executors is not supported by current scheduler.
24/12/01 19:10:48 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$